<a href="https://colab.research.google.com/github/GJHSimmons/personal/blob/main/temperature_dependence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Imports

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import plotly.express as px
import pandas as pd

In [2]:
#@title Temperature model

# Time setup: 1 year, hourly resolution
days = 365
time = np.linspace(0, days)  # time in days

# Temperature parameters
mean_temp = 15               # Mean annual temperature (°C)
annual_amp = 10              # Seasonal amplitude (°C)

# Generate temperature
seasonal = annual_amp * np.sin(2 * np.pi * time / 365)      # seasonal variation
temperature = mean_temp + seasonal

In [3]:
#@title Temperature model plot

fig = px.line(x=time, y=temperature)

fig.show()

In [8]:
#@title Model rate curves

# Temperature lookup for any t
def T(t):
    idx = np.argmin(np.abs(time - t%days))
    return temperature[idx]

# Birth rate curve
def birth_rate(t):
    return np.amax(-0.01*T(t) * (T(t) - 30), 0)

# Death rate curve
def death_rate(t):
    return 0.01*T(t)*(T(t) - 30) + 3

In [9]:
#@title Core model

# ODE
def f(t,y):
    r = birth_rate(t) - death_rate(t)
    return [r*y[0]*(1 - y[0]/100)]

# Initial condition
y0 = [10]

# Time span
t_start = 0
t_end = 365

sol = solve_ivp(f, t_span = [t_start, t_end], y0=y0, dense_output=True)

In [10]:
#@title Plot model simulation

t = np.linspace(t_start, t_end, (t_end - t_start) * 10 + 1)

df = pd.DataFrame(
    {"time": t}
    | {f"y_{i+1}": sol.sol(t)[i] for i in range(len(y0))}
    | {
        "T": [T(day) for day in t],
        "BR": [birth_rate(day) for day in t],
        "DR": [death_rate(day) for day in t],
    }
)

# Plot simulation data
fig = px.line(data_frame=df, x=t, y=[f"y_{i+1}" for i in range(len(y0))])
fig.show()

In [7]:
#@title Plot rate curves
fig = px.line(data_frame=df, x=t, y=["BR", "DR"])
fig.show()